# Imports

In [11]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: pho
"""
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2
import sys
import importlib
from threading import Thread
import time # for time.sleep
from ipygany import PolyMesh, Scene, IsoColor, WarpByScalar
import pyvista as pv
# from pyvista import _vtk
import pyvistaqt as pvqt
import colorcet as cc # Colormaps:
import numpy as np
import pandas as pd
from pathlib import Path
import bqplot.scales
import seaborn as sns

#interactive plotting in separate window
# %matplotlib qt
# %matplotlib inline
# %matplotlib notebook $ # Very bad, don't use this one, obsolite
# %matplotlib widget
# %matplotlib qt

from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.cm import hsv
from matplotlib.colors import Normalize, to_rgba_array, to_hex
from matplotlib.figure import Figure
# import mplcursors
import math # For color map generation
from matplotlib.colors import ListedColormap

from copy import deepcopy

import ipywidgets as widgets
# from PyQt5 import QtWidgets, uic
from pyvistaqt import QtInteractor, MainWindow
# from pyqt6 import QApplication
from IPython.external.qt_for_kernel import QtGui
from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))
from IPython.lib.pretty import pretty, pprint
# from pretty import pprint

# Pandas display options
# pd.set_option('display.max_columns', None)  # or 1000
# pd.set_option('display.max_rows', None)  # or 1000
# pd.set_option('display.max_colwidth', -1)  # or 199
pd.set_option('display.width', 1000)
# pd.set_option('display.max_columns', None)  # or 1000
# pd.set_option('display.max_rows', None)  # or 1000
# pd.set_option('display.max_colwidth', -1)  # or 199
# pd.set_option('display.width', 1000)
pd.set_option('display.min_rows', 30)
pd.set_option('display.max_rows', 50)


# Numpy display options: Very useful, uses the whole width of the screen
np.set_printoptions(edgeitems=3, linewidth=4096, 
    formatter=dict(float=lambda x: "%.3g" % x))
np.core.arrayprint._line_width = 144


# import rich as rich
# from rich import inspect
# from rich import print
# from rich import pretty

from PyQt5.QtWidgets import QApplication
import datetime as dt

## Panel:
import param
import panel as pn
from panel.viewable import Viewer

# from pyphoplacecellanalysis.PhoPositionalData.plotting.mixins.general_plotting_mixins import ActivePlacefieldsPlotting, SinglePlacefieldPlottingExtended


# from panel.interact import interact, interactive, fixed, interact_manual
# from panel import widgets
# pn.extension()
pn.extension('tabulator', 'katex')

# from ipywidgets import interact
# # interact
# @interact


# NeuroPy (Diba Lab Python Repo) Loading
try:
    from neuropy import core
    importlib.reload(core)
except ImportError:
    sys.path.append(r'C:\Users\Pho\repos\NeuroPy') # Windows
    # sys.path.append('/home/pho/repo/BapunAnalysis2021/NeuroPy') # Linux
    # sys.path.append(r'/Users/pho/repo/Python Projects/NeuroPy') # MacOS
    print('neuropy module not found, adding directory to sys.path. \n >> Updated sys.path.')
    from neuropy import core
# from neuropy.core.session.dataSession import SessionConfig, DataSessionLoader, DataSession, processDataSssion

from neuropy.core.session.data_session_loader import DataSessionLoader
from neuropy.core.session.dataSession import DataSession
from neuropy.core.epoch import Epoch
from neuropy.core.epoch import NamedTimerange
from neuropy.core import Laps
from neuropy.core import Position
from neuropy.core import FlattenedSpiketrains
from neuropy.core import Neurons
from neuropy.utils.misc import print_seconds_human_readable
from neuropy.plotting import plot_raster
from neuropy.analyses.placefields import PlacefieldComputationParameters, PfND
from neuropy.analyses.laps import estimate_laps, compute_laps_spike_indicies, estimation_session_laps
from neuropy.plotting.placemaps import plot_all_placefields

## Pho's Custom Libraries:
from pyphocorehelpers.general_helpers import PrettyPrintable, get_arguments_as_optional_dict
from pyphoplacecellanalysis.PhoPositionalData.debug_helpers import debug_print_spike_counts

# import PhoPositionalData as pdp
# from pyphoplacecellanalysis.PhoPositionalData. import load_exported, process_data
from pyphoplacecellanalysis.PhoPositionalData.load_exported import *
# from pyphoplacecellanalysis.PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
from pyphoplacecellanalysis.PhoPositionalData.process_data import *
from pyphoplacecellanalysis.PhoPositionalData.plot_data import *
from pyphoplacecellanalysis.PhoPositionalData.plotting.animations import * # make_mp4_from_plotter
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import plot_laps_2d
from pyphoplacecellanalysis.PhoPositionalData.import_data import * # build_spike_positions_list, build_cellID_reverse_lookup_map
from pyphoplacecellanalysis.PhoPositionalData.analysis.interactive_placeCell_config import InteractivePlaceCellConfig, VideoOutputModeConfig, PlottingConfig, build_configs
from pyphoplacecellanalysis.PhoPositionalData.analysis.interactive_placeCell_config import print_subsession_neuron_differences
from pyphoplacecellanalysis.PhoPositionalData.debug_helpers import *

from PendingNotebookCode import build_placefield_multiplotter, process_by_good_placefields

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# class Stage1(param.Parameterized):

#     basedir = param.Foldername(default=r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53', doc="The session folder path to load")
#     input_data_format = param.Selector(['Kamran', 'Bapun'])
    
#     a = param.Number(default=5, bounds=(0, 10))
#     b = param.Number(default=5, bounds=(0, 10))
    
#     ready = param.Boolean(default=False, precedence=-1)

# #     @param.output(('sess', param.Parameter), ('d', param.Number))
# #     def output(self):
# #         sess = DataSessionLoader.kdiba_old_format_session(basedir)
# #         active_sess_config = sess.config
# #         session_name = sess.name

# #         return 
    
#     @param.depends('basedir', 'input_data_format')
#     def view(self):
#         return pn.Row(pn.widgets.LoadingSpinner(), background='black')
#         # c, d = self.output()
#         return pn.pane.Markdown('${a} * {b} = {c}$\n${a}^{{{b}}} = {d}$'.format(
#             a=self.a, b=self.b, c=c, d=d), style={'font-size': '2em'})

#     def panel(self):
#         return pn.Row(self.param, self.view)


class Stage1(param.Parameterized):
    _default_dirs = {'Kamran': [r'R:\data\KDIBA\gor01\one'],
                      'Bapun'  : [r'R:\data\Bapun\Day5TwoNovel']
                    }
    
    
    e_input_data_changed = param.Event()
    
    basedir = param.FileSelector(default=r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53', doc="The session folder path to load", path=(_default_dirs['Kamran'][0] + r'\**')) # r'R:\data\KDIBA\gor01\one\**'
    input_data_format = param.Selector(list(_default_dirs.keys()), default='Kamran')
    

    a = param.Number(default=5, bounds=(0, 10))

    b = param.Number(default=5, bounds=(0, 10))

    ready = param.Boolean(default=False, precedence=-1)

    @param.depends('input_data_format', watch=True)
    def _update_input_format(self):
        curr_default_dirs = self._default_dirs[self.input_data_format]
        # self.param['basedir'].objects = curr_default_dirs
        self.param['basedir'].path = (curr_default_dirs[0] + r'\**')
        # if self.country not in curr_default_dirs:
        #     self.country = curr_default_dirs[0]


    # @param.output(('c', param.Number), ('d', param.Number))
    # def output(self):
    #     return self.a * self.b, self.a ** self.b
    
    @param.output(('sess', 'DataSession'), ('active_sess_config', param.Dict), {'session_name', param.String})
    def output(self):
        if self.input_data_format == 'Bapun':
            sess = DataSessionLoader.bapun_data_session(self.basedir)
        elif self.input_data_format == 'Kamran':
            sess = DataSessionLoader.kdiba_old_format_session(basedir)
        else:
            raise ValueError
        return sess, sess.config, sess.name

    @param.depends('basedir', 'input_data_format')
    def view(self):
        sess, active_sess_config, session_name = self.output()
        return sess.panel_dataframes_overview()
        # return pn.pane.LaTeX('${a} * {b} = {c}$\n${a}^{{{b}}} = {d}$'.format(
        #     a=self.a, b=self.b, c=c, d=d), style={'font-size': '2em'})

    
    # @param.depends('a', 'b')
    # def view(self):
    #     c, d = self.output()
    #     return sess.panel_dataframes_overview()
    #     return pn.pane.LaTeX('${a} * {b} = {c}$\n${a}^{{{b}}} = {d}$'.format(
    #         a=self.a, b=self.b, c=c, d=d), style={'font-size': '2em'})

    def panel(self):
        return pn.Row(self.param, self.view)

stage1 = Stage1()
stage1.panel()



# stage1 = Stage1()
# pn.panel(stage1.param)
# stage1.panel()

In [ ]:
class Stage2(param.Parameterized):
    """ """
    c = param.Number(default=5, bounds=(0, None))
    exp = param.Number(default=0.1, bounds=(0, 3))
    
    @param.depends('c', 'exp')
    def view(self):
        return pn.pane.LaTeX('${%s}^{%s}={%.3f}$' % (self.c, self.exp, self.c**self.exp),
                             style={'font-size': '2em'})
        # return pn.pane.LaTeX('${%s}^{%s}={%.3f}$' % (self.c, self.exp, self.c**self.exp),
        #                      style={'font-size': '2em'})

    def panel(self):
        return pn.Row(self.param, self.view)

stage2 = Stage2(c=stage1.output()[0])
stage2.panel()


In [ ]:
# Build Output Pipeline:
pipeline = pn.pipeline.Pipeline(debug=True)
pipeline.add_stage('Stage 1', Stage1())
pipeline.add_stage('Stage 2', Stage2)

pipeline

# Load Session Data

In [6]:
# KDiba Old Format:
## Data must be pre-processed using the MATLAB script located here: 
# R:\data\KDIBA\gor01\one\IIDataMat_Export_ToPython_2021_11_23.m
# From pre-computed .mat files:
# 07: 
basedir = r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53'
# # ## 08:
# basedir = r'R:\data\KDIBA\gor01\one\2006-6-08_14-26-15'
sess = DataSessionLoader.kdiba_old_format_session(basedir)
active_sess_config = sess.config
session_name = sess.name

# sess.spikes_df.size # (1014937, 18)
# np.sum(sess.neurons.n_spikes) # 1014937
print('session dataframe spikes: {}\nsession.neurons.n_spikes summed: {}\n'.format(sess.spikes_df.shape, np.sum(sess.neurons.n_spikes)))

## Estimate the Session's Laps data using my algorithm from the loaded position data.
sess = estimation_session_laps(sess)
# ERROR: neuron_type value was of unknown type. type(value[0]): <class 'numpy.int32'>, value[0]: 0, 
#  type(value): <class 'numpy.ndarray'>, value: [0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 2 1 0 0 1 2 1 0 0 1 1 0 1 0 0 0 0 0 1 1 2 2 0 1 0 1 0 0 1 1 0 0 0 0 2]!


	 basepath: R:\data\KDIBA\gor01\one\2006-6-07_11-26-53
	 session_name: 2006-6-07_11-26-53
Loading matlab import file: R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.epochs_info.mat...
done.
Loading matlab import file: R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.position_info.mat...
done.
Loading matlab import file: R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.spikes.mat...
done.
Failure loading .position.npy. Must recompute.

Computing linear positions for all active epochs for session...
curr_active_epoch_timeslice_indicies: Int64Index([   27,    28,    29,    30,    31,    32,    33,    34,    35,    36,
            ...
            52139, 52140, 52141, 52142, 52143, 52144, 52145, 52146, 52147, 52148], dtype='int64', length=52122)
 	 np.shape(curr_active_epoch_timeslice_indicies): (52122,)
curr_active_epoch_timeslice_indicies: Int64Index([52149, 52150, 52151, 52152, 52153, 52154, 52155, 52156, 52157, 52158,
            ...
            

In [ ]:
test_spikes_df = deepcopy(sess.spikes_df)
# test_spikes_df['cell_type'].astype(int)

test_value = test_spikes_df['cell_type'][0]

test_value.name
# test_value.value
# 'NeuronType'
# np.array([a_type.value for a_type in test_spikes_df['cell_type'].values])

test_spikes_df.loc[:, 'cell_type'] = np.array([a_type.value for a_type in test_spikes_df['cell_type'].values])
test_spikes_df

# sess.spikes_df

In [ ]:
# np.min(sess.spikes_df.shank.values)
sess.spikes_df
# sess.panel_dataframes_overview()

### Lap Specific Session Debugging



In [9]:
lap_specific_epochs = sess.laps.as_epoch_obj()
# lap_specific_epochs.to_dataframe()
any_lap_specific_epochs = lap_specific_epochs.label_slice(lap_specific_epochs.labels[np.arange(len(sess.laps.lap_id))])
even_lap_specific_epochs = lap_specific_epochs.label_slice(lap_specific_epochs.labels[np.arange(0, len(sess.laps.lap_id), 2)])
odd_lap_specific_epochs = lap_specific_epochs.label_slice(lap_specific_epochs.labels[np.arange(1, len(sess.laps.lap_id), 2)])

# Filter Session by Epoch:

In [10]:
sess.epochs.t_start = 22.26 # exclude the first short period where the animal isn't on the maze yet
# sess.epochs.to_dataframe()
# active_epoch = sess.epochs.get_named_timerange('maze1')
# print('active_epoch: {}'.format(active_epoch))
# active_epoch = sess.epochs.get_named_timerange('maze2')
active_epoch = NamedTimerange(name='maze', start_end_times=[sess.epochs['maze1'][0], sess.epochs['maze2'][1]])
active_subplots_shape = (1,1) # Single subplot
# active_subplots_shape = '1|2' # 1 subplot on left, two on right
active_config = build_configs(active_sess_config, active_epoch, active_subplots_shape = active_subplots_shape)

## All Spikes:
active_epoch_session = sess.filtered_by_neuron_type('pyramidal').filtered_by_epoch(active_epoch)
print_subsession_neuron_differences(sess.neurons, active_epoch_session.neurons)
# print(sess.neurons.n_spikes)

# # ## Lap_specific Spikes Only:
# active_lap_specific_epoch_session = lap_specific_session.filtered_by_neuron_type('pyramidal').filtered_by_epoch(active_epoch)
# print_subsession_neuron_differences(lap_specific_session.neurons, active_lap_specific_epoch_session.neurons)
# # print(active_lap_specific_epoch_session.neurons.n_spikes)

## Configure Placefield Calc:
should_display_2D_plots = False
should_save_to_disk = True


# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=2, smooth=0.5, frate_thresh=2.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=10, smooth=0.5, frate_thresh=2.0) # works well
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=2.5, smooth=1.5, frate_thresh=2.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=(10, 3), smooth=(0.5, 0.5), frate_thresh=0.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=0.0, grid_bin=(3, 4), smooth=(2, 1), frate_thresh=2.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=(10, 10), smooth=(0.5, 0.5), frate_thresh=2.0) ## Works well for 2D Placemaps
# height: 20.0
# width: 250.0
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=0, grid_bin=(2.0, 0.2), smooth=(0.5, 0.5), frate_thresh=2.0) ## Extremely Slow
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=0, grid_bin=(2.0, 1.0), smooth=(0.5, 0.5), frate_thresh=2.0) ## Very slow, doesn't work

# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=(10, 3), smooth=(0.0, 0.0), frate_thresh=2.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=(10, 3), smooth=(0.1, 0.1), frate_thresh=2.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=(10, 3), smooth=(1.0, 10.0), frate_thresh=2.0)

# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=0.0, grid_bin=(25, 9), smooth=(0.0, 0.0), frate_thresh=2.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=0.0, grid_bin=(5, 3), smooth=(0.0, 0.0), frate_thresh=2.0)
active_config.computation_config = PlacefieldComputationParameters(speed_thresh=0.0, grid_bin=(5, 3), smooth=(0.0, 0.0), frate_thresh=0.1) # TODO: FIXME: BUG: when frate_thresh=0.0, there are 0 good placefield_neuronIDs for all computations!


# active_config.computation_config.frate_thresh = 0.0

NameError: name 'build_configs' is not defined

### TESTING: position binning with _bin_pos_nD testing and validation:

## Compute Placefields if needed:

## Compute the placefields for lap-only filtered Session:

In [ ]:

active_epoch_placefields1D, active_epoch_placefields2D = compute_placefields_masked_by_epochs(active_epoch_session, active_config, included_epochs=None, should_display_2D_plots=should_display_2D_plots)
even_lap_specific_placefields1D, even_lap_specific_placefields2D = compute_placefields_masked_by_epochs(active_epoch_session, active_config, included_epochs=even_lap_specific_epochs, should_display_2D_plots=should_display_2D_plots)
odd_lap_specific_placefields1D, odd_lap_specific_placefields2D = compute_placefields_masked_by_epochs(active_epoch_session, active_config, included_epochs=odd_lap_specific_epochs, should_display_2D_plots=should_display_2D_plots)
any_lap_specific_placefields1D, any_lap_specific_placefields2D = compute_placefields_masked_by_epochs(active_epoch_session, active_config, included_epochs=any_lap_specific_epochs, should_display_2D_plots=should_display_2D_plots)
# Compare the results
# debug_print_ratemap(active_epoch_placefields1D.ratemap)
# num_spikes_per_spiketrain = np.array([np.shape(a_spk_train)[0] for a_spk_train in active_epoch_placefields1D.spk_t])
# num_spikes_per_spiketrain
# print('placefield_neuronID_spikes: {}; ({} total spikes)'.format(num_spikes_per_spiketrain, np.sum(num_spikes_per_spiketrain)))

debug_print_placefield(active_epoch_placefields1D)
debug_print_placefield(any_lap_specific_placefields1D)
debug_print_placefield(even_lap_specific_placefields1D)
debug_print_placefield(odd_lap_specific_placefields1D)

# Get the cell IDs that have a good place field mapping:
active_placefields = deepcopy(any_lap_specific_placefields2D) # not changed this from the default placefields2D object
good_placefield_neuronIDs = np.array(active_placefields.ratemap.neuron_ids) # in order of ascending ID
good_placefield_tuple_neuronIDs = active_placefields.neuron_extended_ids

## Filter by neurons with good placefields only:
good_placefields_session = active_epoch_session.get_by_id(good_placefield_neuronIDs) # Filter by good placefields only, and this fetch also ensures they're returned in the order of sorted ascending index ([ 2  3  5  7  9 12 18 21 22 23 26 27 29 34 38 45 48 53 57])
good_placefields_session
    
pf_sort_ind, pf_colors, pf_colormap, pf_listed_colormap = build_units_colormap(good_placefield_neuronIDs)
active_config.plotting_config.pf_sort_ind = pf_sort_ind
active_config.plotting_config.pf_colors = pf_colors
active_config.plotting_config.active_cells_colormap = pf_colormap
active_config.plotting_config.active_cells_listed_colormap = ListedColormap(active_config.plotting_config.active_cells_colormap)

# active_placefields.ratemap.get_sort_indicies()
good_placefield_neuronIDs

# 2D Plotting

In [ ]:
from neuropy.plotting.ratemaps import plot_ratemap_2D, enumTuningMap2DPlotVariables, enumTuningMap2DPlotMode

curr_fig_resolution_multiplier = 1.0
curr_fig_column_width=8.0
curr_fig_row_height=1.0

plotted_unit_indicies=None # all # good subplots=(10,5)
# curr_subplots=(10,5)
# curr_single_plot_size=(4,1)
# curr_figsize=(30,30)

# plotted_unit_indicies=np.arange(5) # good subplots=(3,3)
# curr_subplots=(3,2)
# curr_figsize=(30,10)

# plotted_unit_indicies=[1] # good subplots=(3,3)
# curr_subplots=(1,1)
# curr_figsize=(30,10)

# plotted_unit_indicies=np.arange(8)
# curr_subplots=(2,3)
# curr_figsize=(30,10)

# plotted_unit_indicies=np.arange(8+6)
# curr_subplots=(2,3)
# curr_figsize=(30,10)

# plotted_unit_indicies=np.arange(8)
# curr_subplots=(10,3)
# curr_figsize=(30,10)
# fig = plt.figure(fignum + fig_ind, figsize=active_figure_size, dpi=None, clear=True, facecolor='r')

# plot_ratemap_2D(active_epoch_placefields2D.ratemap, plot_variable=enumTuningMap2DPlotVariables.FIRING_MAPS, figsize=(30, 30), subplots=(4,20), enable_spike_overlay=True, spike_overlay_spikes=active_epoch_placefields2D.spk_pos)
# plot_ratemap_2D(active_epoch_placefields2D.ratemap, plot_variable=enumTuningMap2DPlotVariables.FIRING_MAPS, figsize=(30, 10), subplots=(2,3), enable_spike_overlay=True, spike_overlay_spikes=active_epoch_placefields2D.spk_pos, included_unit_indicies=np.arange(8))


# plot_ratemap_2D(active_epoch_placefields2D.ratemap, plot_variable=enumTuningMap2DPlotVariables.FIRING_MAPS, plot_mode=enumTuningMap2DPlotMode.IMSHOW, figsize=curr_figsize, subplots=curr_subplots, enable_spike_overlay=True, spike_overlay_spikes=active_epoch_placefields2D.spk_pos, included_unit_indicies=plotted_unit_indicies)
# plot_ratemap_2D(active_epoch_placefields2D.ratemap, plot_variable=enumTuningMap2DPlotVariables.FIRING_MAPS, plot_mode=enumTuningMap2DPlotMode.PCOLORFAST, figsize=curr_figsize, subplots=curr_subplots, enable_spike_overlay=True, spike_overlay_spikes=active_epoch_placefields2D.spk_pos, included_unit_indicies=plotted_unit_indicies)

# plot_ratemap_2D(active_epoch_placefields2D.ratemap, plot_variable=enumTuningMap2DPlotVariables.TUNING_MAPS, plot_mode=enumTuningMap2DPlotMode.IMSHOW, figsize=curr_figsize, subplots=curr_subplots, enable_spike_overlay=True, spike_overlay_spikes=active_epoch_placefields2D.spk_pos, included_unit_indicies=plotted_unit_indicies)
# plot_ratemap_2D(active_epoch_placefields2D.ratemap, plot_variable=enumTuningMap2DPlotVariables.TUNING_MAPS, plot_mode=enumTuningMap2DPlotMode.PCOLORFAST, figsize=curr_figsize, subplots=curr_subplots, enable_spike_overlay=True, spike_overlay_spikes=active_epoch_placefields2D.spk_pos, included_unit_indicies=plotted_unit_indicies)

plot_ratemap_2D(active_epoch_placefields2D.ratemap, fig_column_width=curr_fig_column_width, fig_row_height=curr_fig_row_height, resolution_multiplier=curr_fig_resolution_multiplier, subplots=(None, 3), enable_spike_overlay=True, spike_overlay_spikes=active_epoch_placefields2D.spk_pos, included_unit_indicies=plotted_unit_indicies)

In [ ]:
ax_pf_1D, occupancy_fig, active_pf_2D_figures, active_pf_2D_gs = plot_all_placefields(active_epoch_placefields1D, active_epoch_placefields2D, active_config, should_save_to_disk=should_save_to_disk)

In [ ]:
any_lap_specific_ax_pf_1D, any_lap_specific_occupancy_fig, any_lap_specific_active_pf_2D_figures, any_lap_specific_active_pf_2D_gs = plot_all_placefields(any_lap_specific_placefields1D, any_lap_specific_placefields2D, active_config, variant_identifier_label='any_lap', should_save_to_disk=should_save_to_disk);

In [ ]:
even_lap_specific_ax_pf_1D, even_lap_specific_occupancy_fig, even_lap_specific_active_pf_2D_figures, even_lap_specific_active_pf_2D_gs = plot_all_placefields(even_lap_specific_placefields1D, even_lap_specific_placefields2D, active_config, variant_identifier_label='even_laps', should_save_to_disk=should_save_to_disk);

In [ ]:
odd_lap_specific_ax_pf_1D, odd_lap_specific_occupancy_fig, odd_lap_specific_active_pf_2D_figures, odd_lap_specific_active_pf_2D_gs = plot_all_placefields(odd_lap_specific_placefields1D, odd_lap_specific_placefields2D, active_config, variant_identifier_label='odd_laps', should_save_to_disk=should_save_to_disk);

In [ ]:
from neuropy.plotting.placemaps import plot_occupancy_custom, plot_placefield_occupancy


def test_plot_occupancy(active_placefields, should_overlay_animal_trajectory=True):
    occupancy_fig = plt.Figure(figsize=(10,6))
    occupancy_fig, occupancy_ax = plot_occupancy_custom(active_placefields.occupancy, active_placefields.ratemap.xbin_centers, active_placefields.ratemap.ybin_centers, max_normalized=True, fig=occupancy_fig)
    if should_overlay_animal_trajectory:
        # overlay animal position information to make sure they line up.
        occupancy_ax.scatter(active_placefields.x, active_placefields.y, s=5, c='red', alpha=0.1)
    return occupancy_fig, occupancy_ax


# plot_occupancy_1D(even_lap_specific_placefields1D, True)
# plot_occupancy_1D(even_lap_specific_placefields1D, False)
# plot_placefield_occupancy(even_lap_specific_placefields2D)

active_placefields = deepcopy(active_epoch_placefields2D)
# active_placefields = deepcopy(even_lap_specific_placefields2D)

# occupancy_fig, occupancy_ax = test_plot_occupancy(active_placefields, should_overlay_animal_trajectory=True)
# occupancy_fig, occupancy_ax = test_plot_occupancy(active_placefields, should_overlay_animal_trajectory=False)
# occupancy_fig

plot_placefield_occupancy(active_placefields)

# # fig = plt.Figure()
# fig, axs = plt.subplots(3, 1, figsize=(7, 15))

# # even_lap_specific_placefields1D.occupancy.
# # even_lap_specific_placefields1D.ratemap.xbin_centers
# # np.shape(even_lap_specific_placefields1D.ratemap.xbin) # (214,)
# # even_lap_specific_placefields1D.config.grid_bin_1D # 1
# # np.shape(even_lap_specific_placefields1D.ratemap.xbin_centers) # 213

# even_lap_specific_ax_pf_1D, even_lap_specific_occupancy_fig, even_lap_specific_active_pf_2D_figures = plot_all_placefields(even_lap_specific_placefields1D, even_lap_specific_placefields2D, active_config, variant_identifier_label='even_laps')

# even_lap_specific_placefields1D.plot_ratemaps();
# even_lap_specific_placefields1D.plot_ratemaps()

In [ ]:
## TODO: re-computed the simple spike-count "raw" tuning maps to make sure that the spikes actually fall into the bins with high counts. This will help debug both the computations and the plots.

from neuropy.analyses.placefields import Pf2D
from neuropy.core.ratemap import Ratemap

spk_df = deepcopy(active_epoch_session.spikes_df)

xbin = active_epoch_placefields2D.ratemap.xbin.copy()
ybin = active_epoch_placefields2D.ratemap.ybin.copy()
occupancy = active_epoch_placefields2D.occupancy.copy()

neuron_split_spike_dfs = [spk_df.groupby('aclu').get_group(neuron_id)[['t','x','y','lin_pos']] for neuron_id in active_epoch_session.neuron_ids] # dataframes split for each ID:
firing_maps = np.asarray([Pf2D._compute_tuning_map(neuron_split_spike_dfs[i].x.to_numpy(), neuron_split_spike_dfs[i].y.to_numpy(), xbin, ybin, occupancy, None, should_also_return_intermediate_firing_map=True) for i in np.arange(len(neuron_split_spike_dfs))]) # dataframes split for each ID:
tuning_maps = np.asarray([firing_maps[i] / occupancy for i in np.arange(len(firing_maps))])
ratemap = Ratemap(tuning_maps, xbin=xbin, ybin=ybin, neuron_ids=active_epoch_session.neuron_ids)

## TODO: This is where I should I try to plot the 2D map and the spikes and see if they line up
test_i = 9
fig, ax = plot_occupancy_custom(firing_maps[test_i], xbin, ybin, max_normalized=False)
ax.scatter(neuron_split_spike_dfs[test_i]['x'], neuron_split_spike_dfs[test_i]['y'], s=1, alpha=0.5)

## LOOKS GOOD! They line up well!

In [ ]:
## Check Firing Maps (separate from Occupancy maps)

## TODO: This is where I should I try to plot the 2D map and the spikes and see if they line up
test_i = 9
fig, ax = plot_occupancy_custom(firing_maps[test_i], xbin, ybin, max_normalized=False)
ax.scatter(neuron_split_spike_dfs[test_i]['x'], neuron_split_spike_dfs[test_i]['y'], s=1, alpha=0.5)

In [ ]:
# Group and Plot the different occupancy maps to compare them:
occupany_tabs = pn.Tabs(('All', occupancy_fig))
# Add multiple tabs
occupany_tabs.extend([
    ('Any Laps', any_lap_specific_occupancy_fig),
    ('Even Laps', even_lap_specific_occupancy_fig),
    ('Odd Laps', odd_lap_specific_occupancy_fig)
])

occupany_tabs

In [ ]:
pf_1D_plots_row = pn.Row(ax_pf_1D[0].get_figure(), any_lap_specific_ax_pf_1D[0].get_figure(), even_lap_specific_ax_pf_1D[0].get_figure(), odd_lap_specific_ax_pf_1D[0].get_figure())
pf_1D_plots_tab = pn.Column('# Pf 1D Plots', pf_1D_plots_row)

pf_2D_plots_tab = pn.Column('# Pf 2D Plots', pn.Row(active_pf_2D_figures[0], any_lap_specific_active_pf_2D_figures[0], even_lap_specific_active_pf_2D_figures[0], odd_lap_specific_active_pf_2D_figures[0]))

occupancy_plots_row = pn.Row(occupancy_fig, any_lap_specific_occupancy_fig, even_lap_specific_occupancy_fig, odd_lap_specific_occupancy_fig)
occupancy_plots_tab = pn.Column('# Occupancy Plots', occupancy_plots_row)

tabs = pn.Tabs()
# Add multiple tabs
tabs.extend([
    ('Occupancy', occupancy_plots_tab),
    ('Pf_1d', pf_1D_plots_tab),
    ('Pf_2d', pf_2D_plots_tab)
    
    # ('Text', pn.widgets.TextInput()),
    # ('Color', pn.widgets.ColorPicker())
])

tabs

In [ ]:
# print(np.unique(spk_df['aclu']))
# print(np.unique(spk_df['unit_id']))

curr_neuron_IDX = 3
def plot_ratemap(curr_neuron_IDX = 3):
    fig = Figure(figsize=(10, 6))
    ax = fig.subplots()
    mesh_X, mesh_Y = np.meshgrid(ratemap.xbin, ratemap.ybin)
    pfmap = ratemap.tuning_curves[curr_neuron_IDX]
    curr_pfmap = np.array(pfmap)
    curr_pfmap = np.rot90(np.fliplr(curr_pfmap)) / np.nanmax(curr_pfmap)
    ax.pcolormesh(mesh_X, mesh_Y, curr_pfmap, cmap='jet', vmin=0);
    ax.set_title(f'TuningCurve[{curr_neuron_IDX}]')
    pn.pane.Matplotlib(fig)
    return fig

# plot_ratemap(5)
# pn.interact(plot_ratemap, curr_neuron_IDX=3)
pn.interact(plot_ratemap, curr_neuron_IDX=widgets.IntSlider(start=0,end=ratemap.n_neurons,step=1,value=10))



## Main Spike/Placemap plotting:

# 2D Lap Plotting

### 2D Lap Trajectories Visualization:

## 2D Placefield Plotting:

In [ ]:
# import matplotlib as mpl
# mpl.rcParams.update(mpl.rcParamsDefault)
from pyphoplacecellanalysis.PhoPositionalData.plotting.placefield import plot_1d_placecell_validations
# out_figures_list = plot_1d_placecell_validations(active_epoch_placefields1D, active_config.plotting_config, should_save=True, save_mode='pdf')
# out_figures_list = plot_1d_placecell_validations(active_epoch_placefields1D, active_config.plotting_config, should_save=True, save_mode='separate_files')
out_figures_list = plot_1d_placecell_validations(active_epoch_placefields1D, active_config.plotting_config, modifier_string='lap_only', should_save=False)

In [ ]:
print('Closing created figures.')
for a_fig in out_figures_list:
    plt.close(a_fig)

In [ ]:
out_figures_list = plot_1d_placecell_validations(even_lap_specific_placefields1D, active_config.plotting_config, should_save=False, save_mode='separate_files')

In [ ]:
# active_epoch_placefields2D.plotRaw()
curr_cell_id = 39
fig, ax = active_epoch_placefields2D.plotRaw_v_time(curr_cell_id)
# ax[0].plot(t_both, x_both, 'k', alpha=0.8, linewidth=5)
# ax[0].plot(t_even, x_even, 'g', alpha=0.3, linewidth=3)
# ax[0].plot(t_odd, x_odd, 'r', alpha=0.3, linewidth=3)

In [ ]:
active_epoch_placefields2D.plot_ratemaps_2D(figsize=(12,20), enable_spike_overlay=True)

In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import plot_lap_trajectories_2d
fig, axs, laps_pages = plot_lap_trajectories_2d(sess, curr_num_subplots=len(sess.laps.lap_id), active_page_index=0)
fig.suptitle('Lap Trajectories 2D', fontsize=22)
# fig_out_path = active_config.plotting_config.get_figure_save_path('lap_trajectories_2D').with_suffix('.png')
# fig.savefig(fig_out_path)
plt.show()

# p, axs, laps_pages = plot_lap_trajectories_2d(sess, curr_num_subplots=22, active_page_index=0)
# p, axs, laps_pages = plot_lap_trajectories_2d(sess, curr_num_subplots=22, active_page_index=1)

# 3D Plots

In [ ]:
## 3D Plot Selection
list_3d_plot_types = ['Lap Trajectories', 'Tuning Curves', 'Occupancy', 'Spikes and Activity', 'Testing']

### 3D Lap Trajectories Visualization:

In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import plot_lap_trajectories_3d
p, laps_pages = plot_lap_trajectories_3d(sess, curr_num_subplots=10, active_page_index=1)
p.show()

## Tuning Curves 3D Plot:

In [ ]:
from PhoGui.Panel.panel_placefield import build_all_placefield_output_panels, SingleEditablePlacefieldDisplayConfiguration, ActivePlacefieldsPlottingPanel
from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.InteractivePlaceCellTuningCurvesDataExplorer import InteractivePlaceCellTuningCurvesDataExplorer


try: pActiveTuningCurvesPlotter
except NameError: pActiveTuningCurvesPlotter = None # Checks variable p's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
ipcDataExplorer = InteractivePlaceCellTuningCurvesDataExplorer(active_config, active_epoch_session, active_epoch_placefields2D, active_config.plotting_config.pf_colors, extant_plotter=pActiveTuningCurvesPlotter)
pActiveTuningCurvesPlotter = ipcDataExplorer.plot(pActiveTuningCurvesPlotter) # [2, 17449]

# def update_included_cell_ids(curr_neuron_IDX, show_spikes):
def update_included_cell_Indicies(included_cell_INDEXES, show_spikes=False):
    """ Only used for the old ActivePlacefieldsPlottingPanel-style panel """
    # included_cell_INDEXES = [curr_neuron_IDX]
    # included_cell_INDEXES = [ipcDataExplorer.get_cell_id_and_idx(cell_id=an_included_cell_ID)[0] for an_included_cell_ID in included_cell_ids]
    included_cell_ids = [ipcDataExplorer.get_neuron_id_and_idx(neuron_i=an_included_neuron_idx)[1] for an_included_neuron_idx in included_cell_INDEXES]
    # print(included_cell_ids)
    ipcDataExplorer.update_active_placefields(included_cell_INDEXES) # actives only the placefields that have aclu values (cell ids) in the included_cell_ids array.
    if show_spikes:
        ipcDataExplorer.update_active_spikes(np.isin(ipcDataExplorer.active_session.spikes_df['aclu'], included_cell_ids))
    else:
        ipcDataExplorer.update_active_spikes(np.isin(ipcDataExplorer.active_session.spikes_df['aclu'], []))

    ipcDataExplorer.gui['debug_console_widget'].add_line_to_buffer(f'cell_i: {included_cell_INDEXES}, cell_id: {included_cell_ids}')
    # pn.pane.Matplotlib(fig)

### Build Dynamic Panel Interactive Controls for configuring Placefields:

In [ ]:
class EndButtonPanel(object):
    debug_logging = False
    
    def __init__(self, pf_option_indicies=None, pf_option_selected_values=None, num_pfs=None, update_included_cell_Indicies_callback=None, **params):
        super(EndButtonPanel, self).__init__(**params)
        self.final_update_included_cell_Indicies_callback = None
        if update_included_cell_Indicies_callback is not None:
            if callable(update_included_cell_Indicies_callback):
                self.final_update_included_cell_Indicies_callback = update_included_cell_Indicies_callback

        # assert (self.final_update_included_cell_Indicies_callback is not None), "An update_included_cell_Indicies_callback(x) callback is needed."

    def btn_hide_all_callback(self, event):
        if self.debug_logging:
            print('EndButtonPanel.btn_hide_all_callback(...)')
        ipcDataExplorer.clear_all_spikes_included()
        ipcDataExplorer.update_active_placefields([])
        # self.on_hide_all_placefields()

    def btn_update_active_placefields(self, event):
        if self.debug_logging:
            print('EndButtonPanel.btn_update_active_placefields(...)')
        # updated_pf_options_list_ints = ActivePlacefieldsPlottingPanel.options_to_int(self.cross_selector.value) # convert to ints
        # self.on_update_active_placefields(updated_pf_options_list_ints)
        
    def panel(self):
        # Panel pane and widget objects:
        # Action Buttons:
        self.button_hide_all = pn.widgets.Button(name='Hide All', width_policy='min')
        self.button_hide_all.on_click(self.btn_hide_all_callback)
        self.button_update = pn.widgets.Button(name='Refresh', button_type='primary', width_policy='min')
        self.button_update.on_click(self.btn_update_active_placefields)
        return pn.Column(self.button_hide_all, self.button_update, margin=0, width_policy='min', width=70)
    
def build_panel_interactive_placefield_visibility_controls(ipcDataExplorer):
    out_panels = build_all_placefield_output_panels(ipcDataExplorer)
    end_button_panel_obj = EndButtonPanel()
    end_cap_buttons = end_button_panel_obj.panel()
    out_row = pn.Row(*out_panels, end_cap_buttons, height=120)
    # btn_occupancy_map_visibility = pn.widgets.Button(name='Occupancy Map Visibility', width_policy='min')
    # # btn_occupancy_map_visibility = pn.widgets.Toggle(name='Occupancy Map Visibility', value=ipcDataExplorer.occupancy_plotting_config.isVisible, margin=0, width_policy='min')
    # # btn_occupancy_map_visibility.on_clicks
    # btn_occupancy_map_visibility.on_click(ipcDataExplorer.on_occupancy_plot_update_visibility)
    # # btn_occupancy_map_visibility.on_click(ipcDataExplorer.on_occupancy_plot_config_updated)
    # occupancy_widget = btn_occupancy_map_visibility
    
    occupancy_widget = ipcDataExplorer.occupancy_plotting_config.param
    return pn.panel(pn.Column(out_row, pn.Row(occupancy_widget)))

pane = build_panel_interactive_placefield_visibility_controls(ipcDataExplorer)
pane

In [ ]:
from neuropy.plotting.ratemaps import compute_data_extent, compute_data_aspect_ratio, corner_points_from_extents

extent = compute_data_extent(active_epoch_placefields2D.ratemap.xbin, active_epoch_placefields2D.ratemap.ybin)
xmin, xmax, ymin, ymax = extent
print(f'xmin={xmin}, xmax={xmax}, ymin={ymin}, ymax={ymax}')
# compute_aspect_ratio(active_epoch_placefields2D.ratemap.xbin, active_epoch_placefields2D.ratemap.ybin) # returns (8.0, Width_Height_Tuple(width=240.0, height=30.0))

points = corner_points_from_extents(extent)
# points = corner_points_from_extents(extent)
# points = corner_points_from_extents([23, 260, 124, 154])

# points = corner_points_from_extents([23, 260, 124, 154, 0, 1])
print(f'points: {points}, np.shape(points): {np.shape(points)}') # np.shape(points): (4, 2)


point_x = points[:,0]
point_y = points[:,1]

print(f'point_x: {point_x}, np.shape(point_x): {np.shape(point_x)}') # np.shape(points): (4, 2)

In [ ]:
from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.Mixins.ImagePlaneRendering import ImagePlaneRendering

def plot_3d_image_plotter(active_epoch_placefields2D, image_file=r'output\2006-6-07_11-26-53\maze\speedThresh_0.00-gridBin_5.00_3.00-smooth_0.00_0.00-frateThresh_0.10\pf2D-Occupancy-maze-odd_laps-speedThresh_0.00-gridBin_5.00_3.00-smooth_0.00_0.00-frateThresh_0.png'):
    loaded_image_tex = pv.read_texture(image_file)
    pActiveImageTestPlotter = pvqt.BackgroundPlotter()
    return ImagePlaneRendering.plot_3d_image(pActiveImageTestPlotter, active_epoch_placefields2D.ratemap.xbin, active_epoch_placefields2D.ratemap.ybin, active_epoch_placefields2D.ratemap.occupancy, loaded_image_tex=loaded_image_tex)
    
# Texture from file:
image_file = r'output\2006-6-07_11-26-53\maze\speedThresh_0.00-gridBin_5.00_3.00-smooth_0.00_0.00-frateThresh_0.10\pf2D-Occupancy-maze-odd_laps-speedThresh_0.00-gridBin_5.00_3.00-smooth_0.00_0.00-frateThresh_0.png'
pActiveImageTestPlotter = plot_3d_image_plotter(active_epoch_placefields2D, image_file=image_file)

In [ ]:
# origin = poly.center
# origin[-1] -= poly.length / 3.0
# projected = poly.project_points_to_plane(origin=origin)

# ipcDataExplorer.occupancy_plotting_config.barOpacity = 0.9
ipcDataExplorer.occupancy_plotting_config.isVisible = True
print(ipcDataExplorer.occupancy_plotting_config.to_bars_plot_config_dict())
print(ipcDataExplorer.occupancy_plotting_config.to_labels_plot_config_dict())

In [ ]:
# from pyphoplacecellanalysis.PhoPositionalData.plotting.graphs import plot_3d_binned_bars, plot_point_labels
ipcDataExplorer.plot_occupancy_bars()
pn.panel(ipcDataExplorer.occupancy_plotting_config.param)

In [ ]:
from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.shared_helpers import PlotGroup, PlotGroupWrapper
# vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor, vtkmodules.vtkRenderingCore.vtkActor2D

plot_list = [plotActors['plot_3d_binned_bars_Occupancy']['main'], plotActors['plot_point_labels_OccupancyLabels']]
# plot_list

occupancy_bars_plotGroup = PlotGroupWrapper('3d_Occupancy_bars', {'plot_3d_binned_bars_Occupancy':plotActors['plot_3d_binned_bars_Occupancy']['main'], 'plot_point_labels_OccupancyLabels':plotActors['plot_point_labels_OccupancyLabels']})

# occupancy_bars_plotGroup.blocks.SetVisibility(0)
# for a_block in occupancy_bars_plotGroup.blocks:
# for name in occupancy_bars_plotGroup.blocks.keys():
#     a_block = occupancy_bars_plotGroup.blocks[name]
#     a_block.SetVisibility(0)

# occupancy_bars_plotGroup.plots


# for a_plot_name, a_plot in enumerate(plot_list):
#     a_plot.SetVisibility(0)

occupancy_bars_plotGroup.GetVisibility()
occupancy_bars_plotGroup.SetVisibility(1)
    
# for a_plot_name, a_plot in occupancy_bars_plotGroup.plots.items():
#     a_plot.SetVisibility(1)
    
    
# blocks = None
blocks = pv.MultiBlock()
# blocks.append(plotActors['plot_3d_binned_bars_Occupancy']['main'])
# blocks.append(plotActors['plot_point_labels_OccupancyLabels'])

In [ ]:
ipcDataExplorer.active_tuning_curve_render_configs

In [ ]:
ipcDataExplorer.occupancy_plot_actor

In [ ]:
included_cell_INDEXES = np.array([ipcDataExplorer.get_neuron_id_and_idx(neuron_id=an_included_cell_ID)[0] for an_included_cell_ID in ipcDataExplorer.spikes_df['aclu'].to_numpy()]) # get the indexes from the cellIDs
ipcDataExplorer.spikes_df['neuron_idx'] = included_cell_INDEXES.copy()



# pd.set_op
# pd.set_option('display.max_rows', 24)




## ERROR:
# aclu: 42, unit_id: 40, neuron_idx: 26
# aclu: 42, unit_id: 43, neuron_idx: 28

# np.unique(ipcDataExplorer.spikes_df.aclu) # array([ 2,  3,  4,  5,  7,  8,  9, 10, 11, 12, 14, 17, 18, 21, 22, 23, 24, 25, 26, 27, 28, 29, 33, 34, 38, 39, 42, 44, 45, 46, 47, 48, 53, 55, 57, 58, 61, 62, 63, 64])
        
# ipython




# sort_idx = np.sort(ipcDataExplorer.spikes_df.aclu.to_numpy())

temp = deepcopy(ipcDataExplorer.spikes_df[['aclu','unit_id','neuron_idx']])
# temp.sort_values(by=['aclu'])
temp.sort_values(by=['neuron_idx','aclu'])

for a_neuron_idx in np.unique(temp['neuron_idx']):
    print(temp[temp['neuron_idx'] == a_neuron_idx])
    
    # print(np.unique(temp[temp['neuron_idx'] == a_neuron_idx]))

# np.unique(temp[temp['aclu'] == 2])

In [ ]:
#     # single_pf_output_panels = [build_single_placefield_output_panel(ipcDataExplorer.params.unit_labels[i], ipcDataExplorer.params.pf_colors[:,i]) for i in ipcDataExplorer.visible_tuning_curve_indicies] # only for the currently visible
    
#     hex_colors

# ipcDataExplorer.active_tuning_curve_render_configs

# build_all_placefield_output_panels(ipcDataExplorer)

from PhoGui.Panel.panel_placefield import ActivePlacefieldsPlottingPanel

active_new_pf_panel = ActivePlacefieldsPlottingPanel(np.arange(ipcDataExplorer.num_tuning_curve_plot_actors), ipcDataExplorer.visible_tuning_curve_indicies, num_pfs=len(ipcDataExplorer.visible_tuning_curve_indicies),
                                                     update_included_cell_Indicies_callback=update_included_cell_Indicies)
# active_new_pf_panel.panel()
pf_pane = pn.panel(active_new_pf_panel)
pf_pane

In [ ]:
# server = pf_pane.app('localhost:9999')
# server
# server.show()
# server.stop()

In [ ]:
# ipcDataExplorer.num_tuning_curve_plot_actors
# ipcDataExplorer.params.unit_labels
# ipcDataExplorer.params.keys()

single_pf_output_panels = [build_single_placefield_output_panel(ipcDataExplorer.params.unit_labels[i], ipcDataExplorer.params.pf_colors[:,i]) for i in ipcDataExplorer.visible_tuning_curve_indicies] # only for the currently visible
pn.Row(*single_pf_output_panels, height=120, margin=0, background='black')
# ipcDataExplorer.params.pf_colors
# ipcDataExplorer.params.unit_labels
# ipcDataExplorer.params.unit_labels



### Tuning-Curve Height Spikes Plot Testing:

In [ ]:
# # included_cell_ids = [2, 3, 14, 22, 29, 42, 48, 61]
# included_cell_ids = [14, 39]
# included_cell_INDEXES = [ipcDataExplorer.get_cell_id_and_idx(cell_id=an_included_cell_ID)[0] for an_included_cell_ID in included_cell_ids]
# print(included_cell_INDEXES)




# update_included_cell_ids(5)
# interact(update_included_cell_ids, curr_neuron_IDX=(0,(active_epoch_placefields2D.ratemap.n_neurons-1)))
interact(update_included_cell_ids, curr_neuron_IDX=(0,(active_epoch_placefields2D.ratemap.n_neurons-1)), show_spikes=False)
# interact_manual(update_included_cell_ids, curr_neuron_IDX=widgets.IntSlider(start=0, end=active_epoch_placefields2D.ratemap.n_neurons, step=1, value=10))

In [ ]:
# def func1(x):
#     return 5*x

# # Create a grid around that surface
# grid = pv.create_grid(surface)
# # Clip the grid using the surface
# model = grid.clip_surface(surface)
# # Compute height and display it
# model.elevation().plot()


# interact(func1, x=(0,(active_epoch_placefields2D.ratemap.n_neurons-1)))

# # interact_manual(func1, x=(0,(active_epoch_placefields2D.ratemap.n_neurons-1)))
# # interact(func1, x=IntSlider(min=10, max=50, value=25, step=2, description="Integer Slider"))


    
item_layout = widgets.Layout(height='120px', min_width='40px')
items = [pn.Row(a_widget, layout=item_layout, margin=0, background='black') for a_widget in single_pf_output_panels]

# items = [widgets.Button(layout=item_layout, description=str(i), button_style='success') for i in range(40)]
# build_carousel_scroller(items)
build_carousel_scroller(single_pf_output_panels)


In [ ]:
neuron_idx_int_slider = widgets.IntSlider(min=0, max=(active_epoch_placefields2D.ratemap.n_neurons-1), value=10, step=1, description="Cell Index Int Slider")
neuron_idx_int_slider

In [ ]:
interact(func1, x=neuron_idx_int_slider)

In [ ]:
## Test Hiding/Showing Displayed Tuning Curves for Given CellIDs:
ipcDataExplorer.update_active_placefields(included_cell_INDEXES) # actives only the placefields that have aclu values (cell ids) in the included_cell_ids array.    

In [ ]:
## Test Hiding/Showing Display Spikes for Given CellIDs:
ipcDataExplorer.update_active_spikes(np.isin(ipcDataExplorer.active_session.spikes_df['aclu'], included_cell_ids))

In [ ]:

# np.where(np.logical_not(np.isnan(spike_pf_heights_2D[cell_i]))) # spike_pf_heights_2D: all NaN

# Add a custom z override for the spikes but with the default value so nothing is changed:
ipcDataExplorer.active_session.spikes_df['z'] = np.full_like(ipcDataExplorer.active_session.spikes_df['x'].values, 1.1) # Offset a little bit in the z-direction so we can see it

# set the z values for the current cell index to the heights offset for that cell:
ipcDataExplorer.active_session.spikes_df.loc[(ipcDataExplorer.active_session.spikes_df.aclu == curr_cell_id), 'z'] = spike_pf_heights_2D[cell_i]

ipcDataExplorer.update_spikes()
# ipcDataExplorer.plot()
# ipcDataExplorer.plots['spikes_pf_active'] = ipcDataExplorer.p.add_mesh(historical_spikes_pc, name='spikes_pf_active', scalars='cellID', cmap=ipcDataExplorer.active_config.plotting_config.active_cells_listed_colormap, show_scalar_bar=False, lighting=False, render=False)


# turn on specific pf: (aclu == 14)


In [ ]:
active_epoch_session.position

vectors = [[0, 0, 10.0]] * len(active_epoch_session.spikes_df['x']) # just up in the z-direction
spike_series_positions = active_epoch_session.spikes_df[['x','y']].to_numpy().T    
z_fixed = np.full_like(spike_series_positions[0,:], 1.1) # Offset a little bit in the z-direction so we can see it
origins = np.vstack((spike_series_positions[0,:], spike_series_positions[1,:], z_fixed)).T
# origins

# Perform ray trace
# Define line segment
# start = [0, 0, 0]
# stop = [0.25, 1, 0.5]

curr_idx = 0
curr_tuning_curve_mesh = ipcDataExplorer.plots['tuningCurvePlotActors'][curr_idx]
start = origins[curr_idx]
stop = start + vectors[curr_idx]


## Interactive 3D Spike and Behavior Browser: 

In [ ]:
import PhoGui
from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.PhoInteractivePlotter import PhoInteractivePlotter
from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.shared_helpers import InteractivePyvistaPlotterBuildIfNeededMixin
from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.InteractivePlaceCellDataExplorer import InteractivePlaceCellDataExplorer

active_config.plotting_config.show_legend = True

try: pActiveInteractivePlaceSpikesPlotter
except NameError: pActiveInteractivePlaceSpikesPlotter = None # Checks variable p's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
ipspikesDataExplorer = InteractivePlaceCellDataExplorer(active_config, active_epoch_session, extant_plotter=pActiveInteractivePlaceSpikesPlotter)
pActiveInteractivePlaceSpikesPlotter = ipspikesDataExplorer.plot(pActivePlotter=pActiveInteractivePlaceSpikesPlotter)

## CustomDataExplorer 3D Plotter:

In [ ]:
from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.InteractiveCustomDataExplorer import InteractiveCustomDataExplorer
active_laps_config = InteractivePlaceCellConfig(active_session_config=sess.config, active_epochs=None, video_output_config=None, plotting_config=None) # '3|1    
active_laps_config.plotting_config = PlottingConfig(output_subplots_shape='1|5', output_parent_dir=Path('output', sess.config.session_name, 'custom_laps'))

try: pActiveInteractiveLapsPlotter
except NameError: pActiveInteractiveLapsPlotter = None # Checks variable p's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
iplapsDataExplorer = InteractiveCustomDataExplorer(active_laps_config, sess, extant_plotter=pActiveInteractiveLapsPlotter)
pActiveInteractiveLapsPlotter = iplapsDataExplorer.plot(pActivePlotter=pActiveInteractiveLapsPlotter)

In [ ]:
len(sess.laps.lap_id) # 44

In [ ]:
for i in np.arange(len(sess.laps.lap_id)):
    curr_lap_id = sess.laps.lap_id[i]
    curr_lap_t_start, curr_lap_t_stop = sess.laps.get_lap_times(i)
    curr_lap_position_traces = laps_position_traces[i]
    plot_lap_trajectory_path_spline(iplapsDataExplorer, curr_lap_position_traces, curr_lap_id)

In [ ]:
# p = pv.Plotter(shape=(5, 1), border=True)
p = pActiveInteractiveLapsPlotter
num_laps_to_show = len(sess.laps.lap_id)
num_laps_to_show = 5
flat_lap_id_index = np.arange(len(sess.laps.lap_id))
# subplot_lap_id_index = np.reshape([4,11])
for i in np.arange(num_laps_to_show):
    curr_lap_id = sess.laps.lap_id[i]
    p.subplot(i, 0)
    # curr_lap_t_start, curr_lap_t_stop = sess.laps.get_lap_times(i)
    plot_lap_trajectory_path_spline(iplapsDataExplorer, laps_position_traces[i], curr_lap_id)
    
p.link_views()  # link all the views
p.show()

In [ ]:
# def plot_placefields2D(pTuningCurves, active_placefields, pf_colors: np.ndarray, zScalingFactor=10.0, show_legend=False):

#     # .threshold().elevation()

#     curr_tuning_curves = active_placefields.ratemap.normalized_tuning_curves
#     # curr_tuning_curves[curr_tuning_curves < 0.1] = np.nan
#     curr_tuning_curves = curr_tuning_curves * zScalingFactor

#     num_curr_tuning_curves = len(curr_tuning_curves)
#     # Get the cell IDs that have a good place field mapping:
#     good_placefield_neuronIDs = np.array(active_placefields.ratemap.neuron_ids) # in order of ascending ID
#     tuningCurvePlot_x, tuningCurvePlot_y = np.meshgrid(active_placefields.ratemap.xbin_centers, active_placefields.ratemap.ybin_centers)

#     pdata_currActiveNeuronTuningCurve = pv.StructuredGrid(tuningCurvePlot_x, tuningCurvePlot_y, curr_active_neuron_tuning_Curve)
#     pdata_currActiveNeuronTuningCurve["Elevation"] = curr_active_neuron_tuning_Curve.ravel(order="F")
        
#     return pTuningCurves

# plot_placefields2D(pTuningCurves, active_epoch_placefields

# ipcDataExplorer.plots['spikes_pf_active']['cellID']

# only_active_colormap = ipcDataExplorer.active_config.plotting_config.active_cells_listed_colormap.copy()
# only_active_colormap
test_full_pc_data = ipcDataExplorer.plots_data['spikes_pf_active']['historical_spikes_pc']

# test_subset = test_full_pc_data.GetCellGhostArray()
# test_subset = test_full_pc_data.GetCellData() # vtkmodules.vtkCommonDataModel.vtkCellData
# test_subset = test_full_pc_data[test_full_pc_data['cellID'] == 55] # vtkmodules.vtkCommonDataModel.vtkCellData

# test_subset = test_full_pc_data.GetGhostArray() # [test_full_pc_data['cellID'] == 55] # vtkmodules.vtkCommonDataModel.vtkCellData


unique_ids = np.unique(test_full_pc_data['cellID']) # array([ 0,  1,  2,  3,  5,  7,  9, 10, 15, 16, 19, 20, 21, 22, 25, 26, 27, 31, 32, 36, 37, 40, 42, 43, 44, 45, 46, 51, 53, 55, 56])
# count_arr = np.bincount(test_full_pc_data['cellID'])
count_arr
# array([ 3591,  3311,   231,  4410,     0,   910,     0,  4417,     0,
#         1050,  2443,     0,     0,     0,     0,  2156,  6664,     0,
#            0, 10052,  6286,  2394,  1561,     0,     0,  8827,  4788,
#         6412,     0,     0,     0,   959,  2051,     0,     0,     0,
#         4809,  2758,     0,     0,  1575,     0,  3465, 14861,  2212,
#          777,  2772,     0,     0,     0,     0,  3206,     0,  4032,
#            0,  6489,  2674], dtype=int64)

# good_placefield_neuronIDs: [  5   8  10  13  14  16  19  21  23  25  28  31  32  33  36  37  41  49
#   52  53  54  55  57  59  60  61  62  63  64  66  68  69  74  75  76  78
#   83  86  88  89  90  92  96  98 105 108]; (46 good)

# 1, 4, 6, 7, 8, 11, 18, 19, 21 ## CONCLUSION: the ones that work are indeed the non-zero entries, but the checkboxes work in reverse order to this array that's printed. Meaning you start at the end and work back.
# len(count_arr) # 107
print(unique_ids) # [  3   6   8  11  12  14  17  19  21  23  26  29  30  31  34  35  39  47
  # 50  51  52  53  55  57  58  59  60  61  62  64  66  67  72  73  74  76
  # 81  84  86  87  88  90  94  96 103 106]


# uniques, indicies, inverse_indicies, count_arr = np.unique(active_epoch_session.spikes_df['aclu'].values, return_index=True, return_inverse=True, return_counts=True)
# # count_arr = np.bincount(active_epoch_session.spikes_df['aclu'].values)
# print('active_epoch_session.spikes_df unique aclu values: {}'.format(uniques))
# print('active_epoch_session.spikes_df unique aclu value counts: {}'.format(count_arr))
# print(len(uniques)) # 46 
# uniques, indicies, inverse_indicies, count_arr = np.unique(active_epoch_session.spikes_df['unit_id'].values, return_index=True, return_inverse=True, return_counts=True)
# # count_arr = np.bincount(active_epoch_session.spikes_df['unit_id'].values)
# print('active_epoch_session.spikes_df unique unit_id values: {}'.format(uniques))
# print('active_epoch_session.spikes_df unique unit_id value counts: {}'.format(count_arr))
# print(len(uniques)) # 46 

In [ ]:
# pActiveTuningCurvesPlotter.export_obj('export.obj')
# pActiveTuningCurvesPlotter.export_gltf('export.gltf')

ipcDataExplorer.update_placefield_spike_visibility([38], True) # seems to work

In [ ]:
ipcDataExplorer.update_placefield_spike_visibility([53], True) # seems to work

In [ ]:
ipcDataExplorer.update_placefield_spike_visibility([44, 53], True) # seems to work

# test_subset
# test_subset

In [ ]:
ipcDataExplorer.update_placefield_spike_visibility([53, 44], False) # seems to work

In [ ]:
ipcDataExplorer.gui['tuningCurveSpikeVisibilityCallbacks'][1](False)
# ipcDataExplorer.get_cell_index([2, 3])

In [ ]:
mesh = ipcDataExplorer.plots_data['spikes_pf_active']['historical_spikes_pc'].cast_to_unstructured_grid()
# mesh.n_cells
mesh_unique_ids = np.unique(mesh['cellID'])
print('n_cells: {}'.format(mesh.n_cells)) # 122143
print('cellIDs of mesh: {}'.format(mesh_unique_ids))
# np.unique(mesh['cellID']) # array([ 0,  1,  2,  3,  5,  7,  9, 10, 15, 16, 19, 20, 21, 22, 25, 26, 27, 31, 32, 36, 37, 40, 42, 43, 44, 45, 46, 51, 53, 55, 56])
# ipcDataExplorer.hide_placefield_spikes([0,1,2,3], should_invert=True)
# ipcDataExplorer.hide_placefield_spikes([23,2,34], should_invert=True)

# ipcDataExplorer.hide_placefield_spikes([34], should_invert=True)
ipcDataExplorer.hide_placefield_spikes([38], should_invert=False)

In [ ]:
# the list of spike times that occured for this cell:
i = 2
active_epoch_session.neurons.neuron_ids[i]
active_epoch_session.neurons.spiketrains[i]

# spikes_df = FlattenedSpiketrains.build_spike_dataframe(sess)


In [ ]:
## Timestamp Fixing:
def tt(position_timestamps, t_begin, SampleRate):
    return ((position_timestamps - t_begin) / (1e6 * SampleRate))

# sess.laps.lap_start_stop_flat_idx

sess.recinfo.dat_sampling_rate


# Programmatically change the animal position trail:

In [ ]:
curr_lap_id = int_input.value

# plot_lap_trajectory_path(ipspikesDataExplorer, curr_lap_position_traces)
plot_lap_trajectory_path_spline(ipspikesDataExplorer, curr_lap_position_traces)
# curr_lap_t_start
## TODO: enable showding/hiding the spikes for this data range programmatically in InteractivePlaceCellDataExplorer. 
# ipspikesDataExplorer.
# curr_lap_spike_t_seconds
# curr_lap_spike_indicies
# active_epoch_session.flattened_spiketrains.spikes_df.t_seconds.values[curr_lap_spike_indicies] # index 145937 is out of bounds for axis 0 with size 19647

In [ ]:
# From the laps and position dataframe, extract which lap every position belongs to:

# curr_lap_position_df_is_included = curr_position_df['t'].between(laps_df['start'], laps_df['stop'], inclusive=True) # returns a boolean array indicating inclusion in teh current lap
# curr_lap_position_df = curr_position_df[curr_lap_position_df_is_included] 
# curr_position_df['lap'] = np.NaN

curr_position_df = sess.compute_position_laps()
# curr_position_df.groupby('lap').groups[1] # returns the Int64Index
# curr_position_df.groupby('lap').get_group(2)[['t','x','y','lin_pos']]

lap_specific_position_dfs = [curr_position_df.groupby('lap').get_group(i)[['t','x','y','lin_pos']] for i in sess.laps.lap_id]
lap_specific_position_dfs
curr_lap_position_traces = [lap_pos_df[['x','y']].to_numpy().T for lap_pos_df in lap_specific_position_dfs]
curr_lap_position_traces
curr_lap_time_range = [[lap_pos_df[['t']].to_numpy()[0].item(), lap_pos_df[['t']].to_numpy()[-1].item()] for lap_pos_df in lap_specific_position_dfs]
curr_lap_time_range

In [ ]:
# .GetPosition() returns the actor's position, which is usually (0.0, 0.0, 0.0), and does not relate to the actor's data points
# curr_animal_point = ipspikesDataExplorer.animal_location_trail.GetPosition()
# curr_animal_point = ipspikesDataExplorer.animal_location_trail.GetXRange() # (44.23604202270508, 245.9059600830078)
# print(curr_animal_point)
# curr_animal_point = ipspikesDataExplorer.animal_location_trail.GetYRange() # (135.27638244628906, 145.448974609375)
# print(curr_animal_point)
# curr_animal_point = ipspikesDataExplorer.animal_location_trail.GetZRange() # (1.100000023841858, 1.100000023841858)

curr_animal_point = np.array(ipspikesDataExplorer.animal_location_trail.GetCenter()) # (206.27755737304688, 140.15452575683594, 1.100000023841858)
print('curr_animal_point: {}'.format(curr_animal_point))
curr_animal_point # curr_animal_point

# curr_animal_point = np.array([0, 0, 0])

# curr_animal_point = np.column_stack((self.x[active_window_sample_indicies], self.y[active_window_sample_indicies], self.z_fixed))
# ipspikesDataExplorer.on_programmatic_data_update(curr_animal_point=curr_animal_point)

curr_debug_point = np.array(ipspikesDataExplorer.animal_location_trail.GetCenter()) # (206.27755737304688, 140.15452575683594, 1.100000023841858)
ipspikesDataExplorer.perform_plot_location_point('debug_point_plot', curr_animal_point, color='r')

curr_animal_point = np.array(ipspikesDataExplorer.animal_location_trail.GetCenter()) # not updated
print('new curr_animal_point: {}'.format(curr_animal_point))

In [ ]:
# ipspikesDataExplorer.flattened_spike_positions_list

active_epoch_session.flattened_spiketrains.time_slice(curr_lap_spike_t_seconds.values[0], curr_lap_spike_t_seconds.values[-1]).spikes_df

In [ ]:
curr_lap_dataframe

In [ ]:
active_epoch_session.time_slice

In [ ]:
## Adding interaction highlighters:
pActiveInteractivePlaceSpikesPlotter.